In [ ]:
import shutil
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

model1 = load_model('/Users/ilia/brainTumorClassification/my_model.keras')
model2 = load_model('/Users/ilia/tumorDetection/my_model2.keras')

health_path = '/Users/ilia/DataSetForDiplom/Health'
tumor_path = '/Users/ilia/DataSetForDiplom/Tumor'
anomaly_path = '/Users/ilia/DataSetForDiplom/Anomaly'

for i in range(1, 2081):
    img_path = f'/Users/ilia/tumorDetection/archive-2/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer ({i}).j'
    
    image1 = load_img(img_path, target_size = (150,150))
    image1 = img_to_array(image1)
    image1 = np.expand_dims(image1, axis = 0)
    image1 = image1 / 255.0
    
    image2 = load_img(img_path, target_size = (224,224))
    image2 = img_to_array(image2)
    image2 = np.expand_dims(image2, axis = 0)
    image2 = image2 / 255.0

    predictions1 = model1.predict(image1)
    # print('Number:', i ,'Predictions: ', predictions)
    
    predictions2 = model2.predict(image2)
    # print('Number:', i ,'Predictions: ', predictions1)
    # if predictions1 > 0.5 and  predictions2[0][1] > 0.5:
    #     print('Yes')
    # else:
    #     print('No')

    if predictions1 > 0.5 and predictions2[0][1] > 0.5:
        shutil.move(img_path, tumor_path)
    elif predictions1 <0.5 and predictions2[0][1] < 0.5:
        shutil.move(img_path, health_path)
    else:
        shutil.move(img_path, anomaly_path)